# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
# from api_keys import geoapify_keys
geoapify_keys = '9810630325554d7186141f6bc2b74e8a'
print(geoapify_keys)

9810630325554d7186141f6bc2b74e8a


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olafsvik,64.8945,-23.7142,-2.14,49,100,11.07,IS,1675834465
1,1,bauchi,10.5000,10.0000,16.81,27,80,2.06,NG,1675834868
2,2,cabo san lucas,22.8909,-109.9124,20.28,53,89,0.89,MX,1675834640
3,3,tingo maria,-9.2895,-76.0088,19.89,98,58,0.58,PE,1675834869
4,4,bredasdorp,-34.5322,20.0403,21.14,67,4,5.82,ZA,1675834869


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Display the map
map_plot = city_data_df.hvplot.points('Lng','Lat',geo=True,color='red',alpha=0.2, tiles='OSM', size = "Humidity")

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
desirable_temp_low = 20
desirable_temp_high = 30
desired_city_df = city_data_df.loc[(
    city_data_df["Max Temp"]>desirable_temp_low) 
    & (city_data_df["Max Temp"]<desirable_temp_high
    )]

# Drop any rows with null values
desired_city_df.dropna(inplace=True)

# Display sample data
desired_city_df.head(5)

c:\Users\17372\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,cabo san lucas,22.8909,-109.9124,20.28,53,89,0.89,MX,1675834640
4,4,bredasdorp,-34.5322,20.0403,21.14,67,4,5.82,ZA,1675834869
5,5,busselton,-33.6500,115.3333,28.89,43,0,4.55,AU,1675834870
6,6,sitio novo do tocantins,-5.5183,-47.4611,24.96,88,0,1.03,BR,1675834870
9,9,bambous virieux,-20.3428,57.7575,29.01,74,75,6.69,MU,1675834872


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,olafsvik,IS,64.8945,-23.7142,49,None
1,bauchi,NG,10.5000,10.0000,27,None
2,cabo san lucas,MX,22.8909,-109.9124,53,None
3,tingo maria,PE,-9.2895,-76.0088,98,None
4,bredasdorp,ZA,-34.5322,20.0403,67,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'api_key': geoapify_keys,
    'categories': 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'filter=circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    extra = '&'.join([f'{k}={v}' for k, v in params.items()])
    print(f'{base_url}&q={extra}')
    print()

    # Make and API request using the params dictionaty
    name_address = requests.get(f'{base_url}&q={extra}')

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places&q=api_key=9810630325554d7186141f6bc2b74e8a&categories=accommodation.hotel&filter=filter=circle:-23.7142,64.8945,10000&bias=proximity:-23.7142,64.8945

olafsvik - nearest hotel: No hotel found
https://api.geoapify.com/v2/places&q=api_key=9810630325554d7186141f6bc2b74e8a&categories=accommodation.hotel&filter=filter=circle:10.0,10.5,10000&bias=proximity:10.0,10.5

bauchi - nearest hotel: No hotel found
https://api.geoapify.com/v2/places&q=api_key=9810630325554d7186141f6bc2b74e8a&categories=accommodation.hotel&filter=filter=circle:-109.9124,22.8909,10000&bias=proximity:-109.9124,22.8909

cabo san lucas - nearest hotel: No hotel found
https://api.geoapify.com/v2/places&q=api_key=9810630325554d7186141f6bc2b74e8a&categories=accommodation.hotel&filter=filter=circle:-76.0088,-9.2895,10000&bias=proximity:-76.0088,-9.2895

tingo maria - nearest hotel: No hotel found
https://api.geoapify.com/v2/places&q=api_key=9810630325554d7186141f6bc2b74e

,City,Country,Lat,Lng,Humidity,Hotel Name
0,olafsvik,IS,64.8945,-23.7142,49,No hotel found
1,bauchi,NG,10.5000,10.0000,27,No hotel found
2,cabo san lucas,MX,22.8909,-109.9124,53,No hotel found
3,tingo maria,PE,-9.2895,-76.0088,98,No hotel found
4,bredasdorp,ZA,-34.5322,20.0403,67,No hotel found
...,...,...,...,...,...,...
582,san rafael del sur,NI,11.8485,-86.4384,74,No hotel found
583,coronel fabriciano,BR,-19.5186,-42.6289,94,No hotel found
584,huarmey,PE,-10.0681,-78.1522,84,No hotel found
585,san quintin,MX,30.4833,-115.9500,52,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points('Lng','Lat',geo=True,color='red',alpha=0.2, tiles='OSM', size = "Humidity", hover_cols=['Hotel Name', 'Country'])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)